Run the python script titled 1_table1.ipynb before running this script.

The script uses the table creates in 1_table1.ipynb and prduces the palliative care master table, the palliative care master table shows anyone in Connected Bradford who has had palliative care.
check cell [3] has the latest tables then run script.



In [1]:
# Import libraries.
import pandas as pd
from google.cloud import bigquery
from google.cloud.bigquery import LoadJobConfig
from google.cloud.bigquery import SchemaField

In [2]:
client = bigquery.Client()

/opt/conda/lib/python3.7/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/lib/python3.7/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [3]:
## check the sr_code1 table is the latest from primary care

table_1 = "yhcr-prd-phm-bia-core.CB_FDM_Palliative_care_v1.tmp_table1"
## upload the sr_code1 with the latest tbl_srcode from primary care
sr_code1 = "yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare_V8.tbl_srcode"

The rest of this Script can be run without making any changes

In [4]:
# Load palliative care codelist into a Python pandas data frame
# Plaliative care codes 
palliative_care = pd.read_csv('Palliative_care_codes.csv')
# general palliative care codes
general_care_codes = pd.read_csv('palliative_care_generalist_codes.csv')
#Specialist palliative care codes
specialist_care_codes = pd.read_csv('palliative_care_specialist_codes.csv')

In [5]:
# Delete the current palliative care master table

table_id = 'yhcr-prd-phm-bia-core.CB_FDM_Palliative_care_v1.Palliative_Care_Master'
client.delete_table(table_id, not_found_ok=True)  
print("Deleted table '{}'.".format(table_id))

Deleted table 'yhcr-prd-phm-bia-core.CB_FDM_Palliative_care_v1.Palliative_Care_Master'.


In [6]:
# Join the primary care dataset with the palliative care codes 
# and only show people who have had palliative care in the CBradford dataset.

palliative_care2 = f""" 

with srcode_tbl as (
  select distinct person_id, ctv3code
  from {sr_code1}
)



    SELECT DISTINCT * EXCEPT(Palliative_care_Required)
    from
    
    (select distinct *
    from
    (
    select distinct person_id, datebirth, datedeath, year_of_death, Age_at_death, gender, ethnicity, LSOA, IMD, GP_Practice,
    max(Access_to_palliative_care) as Palliative_care_Required,
    max(Access_to_generalist_palliative_care) as General_PCN, max(Access_to_Specialist_palliative_care) as Specialist_PCN
    
    from
    
    (select distinct a.*, b.* EXCEPT(person_id)
    
    from
    
    (
    select distinct *
    from {table_1}
    )a
    
    left join
    
    (
    SELECT distinct person_id, ctv3code,
        case when ctv3code in ('""" + \
    '\', \''.join(map(str, palliative_care["Code"].tolist())) + \
    """') THEN 1
    ELSE 0
    END AS Access_to_Palliative_care,
    case when ctv3code in ('""" + \
    '\', \''.join(map(str, general_care_codes["Code"].tolist())) + \
    """') THEN 1
    ELSE 0
    END AS Access_to_generalist_palliative_care,
    case when ctv3code in ('""" + \
    '\', \''.join(map(str, specialist_care_codes["Code"].tolist())) + \
    """') THEN 1
    ELSE 0
    END AS Access_to_Specialist_palliative_care 
    
    FROM srcode_tbl
    
    )b
    
    on a.person_id = b.person_id)
    
    group by  person_id, datebirth, datedeath, year_of_death, Age_at_death, gender, ethnicity,LSOA, IMD, GP_Practice
    
    order by person_id asc))
    where Palliative_care_Required = 1
"""


In [7]:
palliative_Care_table2 = pd.read_gbq(palliative_care2)

/opt/conda/lib/python3.7/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [8]:
#first 5 rows of the data
palliative_Care_table2.head()

,person_id,datebirth,datedeath,year_of_death,Age_at_death,gender,ethnicity,LSOA,IMD,GP_Practice,General_PCN,Specialist_PCN
0,908799,1925-06-15,2003-09-15,2003,78,F,Unknown/Refuse to say,E01010655,1,B83064,1,0
1,805470,1927-03-15,1998-09-15,1998,71,F,Unknown/Refuse to say,E01010732,1,B83052,1,0
2,1226301,1944-05-15,2003-08-15,2003,59,F,Black or African or Caribbean or Black British...,None,<NA>,B83010,1,0
3,10417415,1924-10-15,2003-07-15,2003,79,M,Unknown/Refuse to say,E01027555,10,B83033,1,0
4,854745,1931-11-15,1999-10-15,1999,68,M,Unknown/Refuse to say,E01010741,3,B83055,1,0


In [9]:
# drop duplicate rows of data
palliative_care_table2 = palliative_Care_table2.drop_duplicates()

In [10]:
# Create table in Bigquery to store the Palliative care data
final_table = 'yhcr-prd-phm-bia-core.CB_FDM_Palliative_care_v1.Palliative_Care_Master'
# Load the table into BigQuery
job2 = client.load_table_from_dataframe(palliative_care_table2, final_table)

In [12]:
# delete table1 as this isn't needed once the palliative care master table has been created

table_id2 = 'yhcr-prd-phm-bia-core.CB_FDM_Palliative_care_v1.tmp_table1'
client.delete_table(table_id2, not_found_ok=True)  
print("Deleted table '{}'.".format(table_id2))

Deleted table 'yhcr-prd-phm-bia-core.CB_FDM_Palliative_care_v1.tmp_table1'.
